# Exploration of a Double Pendulum's Behavior

## Introduction
A double pendulum consists of a fixed pendulum attached to another pendulum. It is a very simple physical system but it results in chaotic behavior that is sensitive to its initial conditions. 

## Goal 
The goal of this jupyter notebook is to explore the behavior of this system under different sets of controlled initial conditions. Specifically, how will the system evolve over time with different masses with equal rod length or vice versa?

## Assumptions
- There is no energy loss or increase
- The only energies involved are kinetic and potential energies of the two pendulums
- There is no friction
- The rods are massless regardless of length

## Simulation
This simulation will use RK4 integration to solve a coupled first order-differential equation. The fixed initial condition is for the first pendulum to begin at $\pi/2$ while the second pendulum is to be 0 degrees with respect to its pivot point. This simulation will investigate four different scenarios. The first two scenarios will hold lengths constant while the first mass being heavier than the second and the second mass is heavier than the first. The last two scenarios hold the masses constant while differing the lengths between the two pendulums. 

The four first order-differential equations used the pendulums are:

$$ \frac{d\theta_1}{dt} = \omega_1 $$
$$ \frac{d\theta_2}{dt} = \omega_2 $$


$$ \frac{d\omega_1}{dt} = \frac{-g(2m_1+m_2)sin(\theta_1)-m_2gsin(\theta_1-2\theta_2)-2sin(\theta_1-\theta_2)m_2\omega_2^2 L_2 + \omega_1^2 L_1cos(\theta_1-\theta_2))}{L_1(2m_1+ m_2 - m_2cos(2\theta_1 - 2\theta_2))} $$

$$ \frac{d\omega_2}{dt} = \frac{2sin(\theta_1-\theta_2)(\omega_1^2 L_1(m_1+m_2)+g(m_1+m_2)cos(\theta_1) + \omega_2^2 L_2 m_2 cos(\theta_1 - \theta_2))}{L_2(2m_1 +m_2 - m_2cos(2\theta_1 - 2\theta_2))} $$

where $\omega$ is the angular speed and $\frac{d\omega}{dt}$ is the angular acceleration

In [15]:
from vpython import *
import ode
import numpy as np
import matplotlib.pyplot as plt

The function 'dpendulum', handles the calculations of new derivatives for the acceleration and angular speed for each pendulum.

In [16]:
def dpendulum(dep, t):
    """ Calculate and return the derivative [dtheta/dt, domega/dt] evaluated as a function of y, dy/dt, and t
    where y is the dependent variable.
    
    Keyword arguments:
    t -- time at the beginning of the time step
    dep -- an array of the dependent variable and its derivative [y, dy/dt]_n at time t
    """    
    g = 9.8
    
    theta1 = dep[0]
    omega1 = dep[1]
    theta2 = dep[2]
    omega2 = dep[3]
    
    deriv = np.zeros(4)
    
    deriv[0] = omega1 #dtheta/dt
    A1 = -g*(2*m1+m2)*np.sin(theta1)-m2*g*np.sin(theta1-2*theta2)
    A2 = -2*np.sin(theta1-theta2)*m2*((omega2**2)*L2 + (omega1**2)*L1*np.cos(theta1-theta2))
    A3 = L1*(2*m1+ m2 - m2*np.cos(2*theta1 - 2*theta2))
   
    deriv[1] = (A1 + A2)/ A3
    
    deriv[2] = omega2
    
    B1 = 2*np.sin(theta1-theta2)*((omega1**2)*L1*(m1+m2)+g*(m1+m2)*np.cos(theta1) + (omega2**2)*L2*m2*np.cos(theta1-theta2))
    B2 = L2*(2*m1 +m2 - m2*np.cos(2*theta1 - 2*theta2))
    
    deriv[3] = B1/B2
    
    return deriv


The function 'pcalc', takes the initial conditions for the mass and length of each pendulum. Once the values for $\theta$ and $\omega$ have been calculated, the arrays of the corresponding pendulums angles, angular speeds, as well as time, are returned.

In [17]:
def pcalc(L1,m1,L2,m2):
    
    #dependent variables
    theta1_0 = np.pi/2
    theta2_0 = 0
    omega1_0 = 0 #initial speed of theta1
    omega2_0 = 0

    data = np.array([theta1_0, omega1_0, theta2_0, omega2_0]) #initialize array to store dependent variables

    #independent variable
    t = 0
    h = 1e-4
    Nsteps = int(10/h) #N steps for T seconds of evolution

    #create arrays needed for plotting theta vs. t and omega vs. t
    tarr = np.zeros(Nsteps)
    theta1arr = np.zeros(Nsteps)
    theta2arr = np.zeros(Nsteps)

    omega1arr = np.zeros(Nsteps)
    omega2arr = np.zeros(Nsteps)

    tarr[0] = t
    theta1arr[0] = theta1_0
    theta2arr[0] = theta2_0

    omega1arr[0] = omega1_0
    omega2arr[0] = omega2_0

    #create a time evolution loop

    for n in range(1,Nsteps):

        data = ode.RK4(dpendulum, data, t, h)

        t = t + h

        tarr[n] = t
        theta1arr[n] = data[0]
        omega1arr[n] = data[1]
        theta2arr[n] = data[2]
        omega2arr[n] = data[3]
        
    return theta1arr, omega1arr, theta2arr, omega2arr, tarr


The 'animation' function accepts the $\theta$ arrays for each pendulum as well as their massess. The animation is then created and displayed.

In [18]:
def animation(p1, m1, p2, m2): 
    scene = canvas(title="Double Pendulum")


    x1array = L1*np.sin(p1)
    y1array = -L1*np.cos(p1)


    x2array = x1array + L2*np.sin(p2)
    y2array = y1array - L2*np.cos(p2)


    ball1 = sphere(pos = vec(x1array[0],y1array[0],0), radius = m1*.05, make_trail = True, color = color.blue)
    rod1 = cylinder(pos = vec(0,0,0),radius = 2*.025, axis = ball1.pos)

    ball2 = sphere(pos = vec(x2array[0],y2array[0],0), radius = m2*.05, make_trail = True, color = color.red)
    rod2 = cylinder(pos = ball1.pos ,radius = 2*.025, axis = ball2.pos - ball1.pos)
    
    h = 1e-4
    Nsteps = int(10/h)
    
    scene.pause()

    for n in range(1,Nsteps):
        rate(10000)
        ball1.pos = vec(x1array[n],y1array[n],0)
        rod1.axis = ball1.pos

        ball2.pos = vec(x2array[n],y2array[n],0)
        rod2.pos = ball1.pos
        rod2.axis = ball2.pos - ball1.pos


## Equal Lengths ($L_1 = L_2$), $M_1$ larger than $M_2$

In [19]:
L1 = 4
L2 = 4
m1 = 8
m2 = 2

th1,om1,th2,om2,time = pcalc(L1,m1,L2,m2)

In [20]:
animation(th1, m1, th2, m2)

<IPython.core.display.Javascript object>

## Equal Lengths ($L_1 = L_2$), $M_2$ larger than $M_1$

In [21]:
L1 = 4
L2 = 4
m1 = 2
m2 = 8

th1,om1,th2,om2,time = pcalc(L1,m1,L2,m2)

In [22]:
animation(th1, m1, th2, m2)

<IPython.core.display.Javascript object>

## Equal Masses ($M_1 = M_2$), $L_1$ larger than $L_2$

In [23]:
L1 = 8
L2 = 2
m1 = 4
m2 = 4

th1,om1,th2,om2,time = pcalc(L1,m1,L2,m2)

In [24]:
animation(th1, m1, th2, m2)

<IPython.core.display.Javascript object>

## Equal Masses ($M_1 = M_2$), $L_2$ larger than $L_1$

In [25]:
L1 = 2
L2 = 8
m1 = 4
m2 = 4

th1,om1,th2,om2,time = pcalc(L1,m1,L2,m2)

In [ ]:
animation(th1, m1, th2, m2)

<IPython.core.display.Javascript object>

## Conclusion

This simulation is versatile so that the double pendulum initial conditions can be modified with ease. After running the simulation four times, we can see that the behavior of the system varies wildly when changing the masses and lengths. 

## Citations
Differential Equations - https://www.myphysicslab.com/pendulum/double-pendulum-en.html 